In [11]:
import os
import urllib.request
import json
import requests

def asr_from_yandex_speeckit(FOLDER_ID, API_KEY, audiofile_name,
                                language="ru-RU",
                                 ):
    
    with open(audiofile_name, "rb") as f:
        data = f.read()

    params = "&".join([
        "topic=general",
        "folderId=%s" % FOLDER_ID,
        "lang=%s" % language
    ])

    url = urllib.request.Request("https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?%s" % params, data=data)
    url.add_header("Authorization", "Api-Key %s" % API_KEY) # если API-ключ
    # url.add_header("Authorization", "Bearer %s" % IAM_TOKEN) # если IAM токен

    responseData = urllib.request.urlopen(url).read().decode('UTF-8')
    decodedData = json.loads(responseData)

    if decodedData.get("error_code") is None:
        asr_result = decodedData.get("result")
    else:
        asr_result = 'Nothing recognized'
    
    return asr_result

def synthesize(folder_id, api_key, text):
    url = 'https://tts.api.cloud.yandex.net/speech/v1/tts:synthesize'
    headers = {
        'Authorization': 'Api-Key ' + api_key,
    }

    data = {
        'text': text,
        'lang': 'ru-RU',
        'voice': 'filipp',
        'folderId': folder_id
    }

    with requests.post(url, headers=headers, data=data, stream=True) as resp:
        if resp.status_code != 200:
            raise RuntimeError("Invalid response received: code: %d, message: %s" % (resp.status_code, resp.text))

        
        for chunk in resp.iter_content(chunk_size=None):
            yield chunk
            

def tts_from_yandex_speechkit(text, FOLDER_ID, API_KEY, tts_ogg_name="tts_result.ogg"):

    with open(tts_ogg_name, "wb") as f:
        for audio_content in synthesize(FOLDER_ID, API_KEY, text):
            f.write(audio_content)


In [5]:
FOLDER_ID = "b1gami13b761380nb5hb" # Идентификатор каталога
API_KEY = "AQVN3XofRyHEW5PJOhBYCiObnGaZNYM8IvAQskdp" # API-ключ

In [6]:
ogg_name = "/home/appuser/test.ogg"

asr_result = asr_from_yandex_speeckit(FOLDER_ID, API_KEY, audiofile_name=ogg_name)

asr_result

'1 2 3 4'

In [12]:
# import os
# port = 8008

# curl -X POST "http://localhost:8008/asr"  -F 'file=@/home/appuser/test.ogg'

In [60]:
tts_ogg_name = '/home/appuser/tts_result.ogg'
text = "Уничтожение человечества хахаахх"

tts_from_yandex_speechkit(text, FOLDER_ID, API_KEY)

In [10]:
import requests
tts_ogg_name = '/home/appuser/src/test.ogg'
#asr-fastapi test
url = "http://localhost:8008/asr"
files = {'file': open(tts_ogg_name, 'rb')}
r = requests.post(url, files=files)
r.text

'{"transcript":"1 2 3 4"}'

In [2]:
#tts-fastapi test
tts_ogg_name = '/home/appuser/tts_result.ogg'
text = "Привет дивный мир"
files = {'response': (None, text)}
response = requests.post('http://localhost:8008/tts', files=files)
with open(tts_ogg_name, 'wb') as f:
    f.write(response.content) 